In [1]:
from tensorflow.keras.models import load_model
import pickle

# Load model
model = load_model("lstm_hinglish_sentiment.h5")

# Load tokenizer
with open("tokenizer.pkl", "rb") as handle:
    tokenizer = pickle.load(handle)

In [6]:
from tensorflow.keras.preprocessing.sequence import pad_sequences
import numpy as np

max_len = 50  # use the same max_len as during training

import re
from tensorflow.keras.preprocessing.sequence import pad_sequences

# Text cleaning (you can expand this)
def clean_text(text):
    text = text.lower()
    text = re.sub(r"http\S+", "", text)   # remove URLs
    text = re.sub(r"[^a-zA-Z\u0900-\u097F\s]", "", text)
    text = re.sub(r"\s+", " ", text).strip()
    print(text)
    return text

# Function to preprocess and prepare input for model
def preprocess_text(text, tokenizer, max_len):
    cleaned = clean_text(text)
    seq = tokenizer.texts_to_sequences([cleaned])
    padded = pad_sequences(seq, maxlen=max_len, padding='post', truncating='post')
    return padded


In [7]:
def predict_sentiment(text):
    padded_seq = preprocess_text(text, tokenizer, max_len)
    pred = model.predict(padded_seq)
    label = np.argmax(pred, axis=1)[0]
    
    sentiment_map = {0: "Negative", 1: "Neutral", 2: "Positive"}
    return sentiment_map[label]

# Example
user_input = "वैसे मैं sure नहीं हूं कि मैं Metacritic के 70/100 score से agree करूंगा"
print("Predicted Sentiment:", predict_sentiment(user_input))


वैसे मैं sure नहीं हूं कि मैं metacritic के score से agree करूंगा
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 51ms/step
Predicted Sentiment: Positive


In [8]:
from textblob import TextBlob

def baseline_textblob_sentiment(text):
    polarity = TextBlob(text).sentiment.polarity
    if polarity > 0.1:
        return "Positive"
    elif polarity < -0.1:
        return "Negative"
    else:
        return "Neutral"

print("Baseline (TextBlob):", baseline_textblob_sentiment(user_input))
print("LSTM Prediction:", predict_sentiment(user_input))


Baseline (TextBlob): Positive
वैसे मैं sure नहीं हूं कि मैं metacritic के score से agree करूंगा
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 52ms/step
LSTM Prediction: Positive
